In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('wine_quality.csv')
print(df.columns)


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


In [ ]:
y = df['quality']
X = df.drop(columns = ['quality'])

In [ ]:
## 1. Data transformation
from sklearn.preprocessing import StandardScaler

In [ ]:
## 2. Train-test split
from sklearn.model_selection import train_test_split

In [ ]:
## 3. Fit a logistic regression classifier without regularization
from sklearn.linear_model import LogisticRegression

In [ ]:
## 4. Plot the coefficients

In [ ]:
## 5. Training and test performance
from sklearn.metrics import f1_score

In [ ]:
## 6. Default Implementation (L2-regularized!)

## 7. Ridge Scores

In [ ]:
## 8. Coarse-grained hyperparameter tuning
training_array = []
test_array = []
C_array = [0.0001, 0.001, 0.01, 0.1, 1]

In [ ]:
## 9. Plot training and test scores as a function of C

In [ ]:
## 10. Making a parameter grid for GridSearchCV

In [ ]:
## 11. Implementing GridSearchCV with l2 penalty
from sklearn.model_selection import GridSearchCV

In [ ]:
## 12. Optimal C value and the score corresponding to it

In [ ]:
## 13. Validating the "best classifier"

In [ ]:
## 14. Implement L1 hyperparameter tuning with LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
## 15. Optimal C value and corresponding coefficients

In [ ]:
## 16. Plotting the tuned L1 coefficients